<a href="https://colab.research.google.com/github/Gopichand14052004/BATCH_5_NLP/blob/main/assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

preprocessing of data and dividing the data

In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.datasets import imdb

# Load dataset (Using IMDB for text generation)
vocab_size = 10000
max_length = 100

# Load IMDB dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences to ensure uniform input size
X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

# Display dataset shape
print(f'Training data shape: {X_train.shape}')
print(f'Testing data shape: {X_test.shape}')


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training data shape: (25000, 100)
Testing data shape: (25000, 100)


build gru

In [2]:
# Define the GRU model
def build_gru_model(vocab_size, embedding_dim=64, input_length=max_length):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=input_length),
        tf.keras.layers.GRU(128, return_sequences=False),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Initialize and summarize the GRU model
gru_model = build_gru_model(vocab_size)
gru_model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

training gru

In [3]:
# Train the GRU model
gru_history = gru_model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=64)


Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 121s 298ms/step - accuracy: 0.6837 - loss: 0.5561 - val_accuracy: 0.8479 - val_loss: 0.3513
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 145s 305ms/step - accuracy: 0.8946 - loss: 0.2672 - val_accuracy: 0.8408 - val_loss: 0.3667
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 137s 293ms/step - accuracy: 0.9277 - loss: 0.1914 - val_accuracy: 0.8596 - val_loss: 0.3409
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 119s 303ms/step - accuracy: 0.9568 - loss: 0.1244 - val_accuracy: 0.8466 - val_loss: 0.4507
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 132s 338ms/step - accuracy: 0.9744 - loss: 0.0752 - val_accuracy: 0.8427 - val_loss: 0.5283


txt generation using trained model

In [17]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample text data (used for training and fitting the tokenizer)
text_data = [
    "Once upon a time in a faraway land there lived a king",
    "The king had three sons who were brave and kind",
    "They went on adventures through forests and mountains"
]

# Initialize and fit the Tokenizer on the training text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)

# Parameters for text generation
seed_text = "Once upon a time"
next_words = 15  # Number of words to generate
max_length = 100  # Max length used during model training (adjust as per your model)

# Assuming `gru_model` is the pre-trained GRU model for text generation
for _ in range(next_words):
    # Convert the current seed text to a sequence of integers (tokens)
    token_list = tokenizer.texts_to_sequences([seed_text])[0]

    # Pad the sequence to the correct length (max_length - 1)
    token_list = pad_sequences([token_list], maxlen=max_length - 1, padding='pre')

    # Predict the next word's token using the GRU model
    predicted = np.argmax(gru_model.predict(token_list), axis=-1)

    # Find the word corresponding to the predicted token
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break

    # Append the predicted word to the seed text
    seed_text += " " + output_word

# Print the final generated text
print(seed_text)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Once upon a time               


In [18]:
# Evaluate GRU model accuracy
gru_loss, gru_accuracy = gru_model.evaluate(X_test, y_test)
print(f"GRU Model Accuracy: {gru_accuracy * 100:.2f}%")


782/782 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - accuracy: 0.8399 - loss: 0.5411
GRU Model Accuracy: 84.27%


In [8]:
# Define the LSTM model
def build_lstm_model(vocab_size, embedding_dim=64, input_length=max_length):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=input_length),
        tf.keras.layers.LSTM(128, return_sequences=False),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Initialize and summarize the LSTM model
lstm_model = build_lstm_model(vocab_size)
lstm_model.summary()

# Train the LSTM model
lstm_history = lstm_model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=64)

# Evaluate LSTM model accuracy
lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test, y_test)
print(f"LSTM Model Accuracy: {lstm_accuracy * 100:.2f}%")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 116s 290ms/step - accuracy: 0.7068 - loss: 0.5340 - val_accuracy: 0.8500 - val_loss: 0.3416
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 144s 296ms/step - accuracy: 0.8942 - loss: 0.2615 - val_accuracy: 0.8500 - val_loss: 0.3575
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 114s 293ms/step - accuracy: 0.9280 - loss: 0.1921 - val_accuracy: 0.8443 - val_loss: 0.3919
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 153s 321ms/step - accuracy: 0.9519 - loss: 0.1312 - val_accuracy: 0.8322 - val_loss: 0.4589
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 128s 287ms/step - accuracy: 0.9663 - loss: 0.0970 - val_accuracy: 0.8312 - val_loss: 0.5871
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 52ms/step - accuracy: 0.8300 - loss: 0.6007
LSTM Model Accuracy: 83.12%


In [9]:
print(f"GRU Model Accuracy: {gru_accuracy * 100:.2f}%")
print(f"LSTM Model Accuracy: {lstm_accuracy * 100:.2f}%")


GRU Model Accuracy: 84.27%
LSTM Model Accuracy: 83.12%
